# `Technical Test Crypto.com, 14/10/2023`
*Hugo Morel*

### Librairies

In [7]:
import pandas as pd
from pandas.tseries.holiday import *
import numpy as np
import threading
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from random import seed
from random import randint
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Display All dataframe columns
pd.set_option('display.max_columns', None)

In [9]:
# Desable scrolling

In [10]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

<IPython.core.display.Javascript object>

## `Q2 - Design an arbitrage strategy on LINK/USDT`

### Load Data

In [11]:
# bnc Data
bar_bnc                                             = pd.read_csv('../bar_bnc.csv')
orderbook_bnc                                       = pd.read_csv('../orderbook_bnc.csv')
# Set 'timestamp' as index
bar_bnc.set_index('timestamp',inplace = True)
orderbook_bnc.set_index('timestamp',inplace = True)
# hb Data
bar_hb                                              = pd.read_csv('../bar_hb.csv')
orderbook_hb                                        = pd.read_csv('../orderbook_hb.csv')
# Set 'timestamp' as index
bar_hb.set_index('timestamp',inplace = True)
orderbook_hb.set_index('timestamp',inplace = True)

### `1. Design an arbitrage strategy on LINK/USDT between Binance and Huobi based on provided data.`

### Analysis HB and BNC data

#### Analysis Orderbook Data

In [104]:
# Timestamp match between Binance and Huobi Orderbooks
if not False in pd.Series(orderbook_hb.index.values == orderbook_bnc.index.values).value_counts().keys():
    print('Result : Timestamps match between Binance and Huodi Orderbookds')
else:
    print('Result : Timestamps do not match between Binance and Huodi Orderbookds')

Result : Timestamps match between Binance and Huodi Orderbookds


In [116]:
# Binance and Huobi Orderbook Match
if orderbook_hb.shape == orderbook_bnc.shape:
    print('Result : Orderbooks shape are matching')
else:
    print('Result : Orderbooks shape do not match')

Result : Orderbooks shape are matching


In [111]:
# Binance and Huobi Orderbook Columns Match
if not False in (orderbook_hb.columns == orderbook_bnc.columns):
    print('Result : Columns match between datasets')
else:
    print('Result : Columns do not match between datasets')

Result : Columns match between datasets


In [113]:
# No null value into Huobi Orderbook
if orderbook_hb[orderbook_hb.isnull() == False].shape == orderbook_hb.shape:
    print('Result : No null values in Huobi dataset')
else:
    print('Result : There are null values in Huobi dataset')

Result : No null values in Huobi dataset


In [115]:
# No null value into Binance Orderbook
if orderbook_bnc[orderbook_bnc.isnull() == False].shape == orderbook_bnc.shape:
    print('Result : No null values in Binance dataset')
else:
    print('Result : There are null values in Binance dataset')

Result : No null values in Binance dataset


#### Analysis Bar Data

In [131]:
# Only Link/USDT is filled into data
if bar_hb['symbol'].unique()[0][:9] == bar_bnc['symbol'].unique()[0][:9]:
    print('Resut : Only Link/USDT is filled in our data')
else:
    print('Resut : Not only Link/USDT is filled in our data')

Resut : Only Link/USDT is filled in our data


In [124]:
# Binance and Huobi Bat do not Match, more value into bar_bnc
if bar_hb.shape == bar_bnc.shape:
    print('Result : Bar data shapes match')
elif bar_bnc.shape[0] > bar_hb.shape[0]: 
    print('Result : More Bar date in Binance dataset')
else:
    print('Result : More Bar date in Huobi dataset')

Result : More Bar date in Binance dataset


In [128]:
# Binance and Huobi Bar Columns Match
if not False in bar_hb.columns == bar_bnc.columns:
    print('Result : Binance and Huobi bar columns match')
else:
    print('Result : Binance and Huobi bar columns do not match')

Result : Binance and Huobi bar columns match


In [130]:
# No null value into Huobi bar
if bar_hb[bar_hb.isnull() == False].shape == bar_hb.shape:
    print('Result : No null values in Huobi dataset')
else:
    print('Result : There are null values in Huobi dataset')

Result : No null values in Huobi dataset


In [129]:
# No null value into Binance bar
if bar_bnc[bar_bnc.isnull() == False].shape == bar_bnc.shape:
    print('Result : No null values in Binance dataset')
else:
    print('Result : There are null values in Binance dataset')

Result : No null values in Binance dataset


### `2. Code a backtesting program by yourself with Python to test your strategy. Open source backtesting framework is not allowed.`

In [12]:
class Backtesting:
    
    def __init__(self,orderbook_hb,orderbook_bnc,bar_hb,bar_bnc, scenario):
        # Load Binance and Huobi Oderbooks
        self.orderbook_hb       = orderbook_hb
        self.orderbook_bnc      = orderbook_bnc
        
        # Load History price
        self.bar_bnc            = bar_bnc
        self.bar_hb             = bar_hb
        
        # Select scenario
        self.scenario = scenario
        
        # Set fees; % to real value
        self.Fees               = 0
        self.Ask_Fees           = (1+self.Fees)
        self.Bid_Fees           = (1-self.Fees)
        self.Fees_increment     = 0.00001
        
        # Set Columns name of data_Arbitrage
        self.columns                 = ['timestamp','Ask','Bid','By on','Profits by Link','Volume','Total Profits in USDT']
        
        # Create data_arbitrage
        self.data_arbitrage     = pd.DataFrame(columns=self.columns)
        
        
        # Save Epocs Results
        self.epocs = []
        self.epocs_fees = []
        self.epocs_sharpe_ratio = []
        self.epocs_sortino_ratio = []
        
        
        # Risk Metrics
        self.risk_free_rate = 0.0000009645062
        
        
    def run_epocs_fees_limit(self):
        run = True
        
        while run==True:
            res = self.BacktestFunction()
            
            if res.empty:
                run = False
            else:
                self.epocs.append(res)
                self.epocs_fees.append(self.Fees)
                
                sharpe_ratio = self.Compute_sharpe_Ratio(res)
                
                sortino_Ratio = self.Compute_sortino_Ratio(res)
                
                
                self.epocs_sharpe_ratio.append(sharpe_ratio)
                self.epocs_sortino_ratio.append(sortino_Ratio)
        
                print("Fees : ", self.Fees, '- Profit :', self.epocs[-1]['Cumulative of Total Profits in USDT'][len(self.epocs[-1])-1])
            
                self.Fees               = self.Fees + self.Fees_increment
                self.Ask_Fees           = (1+self.Fees)
                self.Bid_Fees           = (1-self.Fees)
            
        return self.epocs
    
    
    def run_risk_metrics(self,res):
        sharpe_ratio = self.Compute_sharpe_Ratio(res)
        
        sortino_Ratio = self.Compute_sortino_Ratio(res)
        
        print("Sharpe Ratio : ",sharpe_ratio)
        print("Sortino Ratio : ",sortino_Ratio)
        
    def Compute_sharpe_Ratio(self, res):
        
        returns_strategy = [ (res['Cumulative of Total Profits in USDT'].iloc[i+1] - res['Cumulative of Total Profits in USDT'].iloc[i]) / res['Cumulative of Total Profits in USDT'].iloc[i] for i in range(0, len(res)-1)]
        
        if len(returns_strategy) < 7200:
            returns_strategy = np.concatenate((returns_strategy, np.zeros(7200-len(returns_strategy))), axis=0)
        
        mean_return = np.mean(returns_strategy)
        std_return = np.std(returns_strategy, axis=0)
        
        
        sharpe_ratio = (mean_return - self.risk_free_rate) / std_return
        
        
        return sharpe_ratio
    
    
    def Compute_sortino_Ratio(self,res):
        returns_strategy = [ (res['Cumulative of Total Profits in USDT'].iloc[i+1] - res['Cumulative of Total Profits in USDT'].iloc[i]) / res['Cumulative of Total Profits in USDT'].iloc[i] for i in range(0, len(res)-1)]
        
        returns_strategy = np.array(returns_strategy)
        expected_return = np.mean(returns_strategy)
        
        downside_returns = returns_strategy[returns_strategy < 0]
        downside_deviation = np.std(downside_returns)
        
        sortino_ratio = (expected_return - self.risk_free_rate) / downside_deviation
        
        return sortino_ratio
        
    def BacktestFunction(self):
        
        # Reset dataframe for epoc
        self.data_arbitrage = pd.DataFrame(columns=self.columns)
        
        for timestamp in self.orderbook_hb.index.values:
            # --- GET HB DATA ---
            # get Ask price
            self.Ask_hb              = orderbook_hb.loc[timestamp][['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10']]
            # get Ask Volume
            self.Ask_Volume_hb       = orderbook_hb.loc[timestamp][['av1','av2','av3','av4','av5','av6','av7','av8','av9','av10']]
            # get Bid price
            self.Bid_hb              = orderbook_hb.loc[timestamp][['b1','b2','b3','b4','b5','b6','b7','b8','b9','b10']]
            # get Bid Volume
            self.Bid_Volume_hb       = orderbook_hb.loc[timestamp][['bv1','bv2','bv3','bv4','bv5','bv6','bv7','bv8','bv9','bv10']]
            
            # --- GET BNC DATA ---
            # get Ask price
            self.Ask_bnc             = orderbook_bnc.loc[timestamp][['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10']]
            # get Ask Volume
            self.Ask_Volume_bnc      = orderbook_bnc.loc[timestamp][['av1','av2','av3','av4','av5','av6','av7','av8','av9','av10']]
            # get Bid price
            self.Bid_bnc             = orderbook_bnc.loc[timestamp][['b1','b2','b3','b4','b5','b6','b7','b8','b9','b10']]
            # get Bid Volume
            self.Bid_Volume_bnc      = orderbook_bnc.loc[timestamp][['bv1','bv2','bv3','bv4','bv5','bv6','bv7','bv8','bv9','bv10']]
            
            # --- RECURSIVE FUNCTIONS
            # Huobi to Binance Recursive function
            self.Check_Arbitrage_Hb_To_Bnc(0,0,timestamp)
            # Binance to Huobi Recursive function
            self.Check_Arbitrage_Bnc_To_Hb(0,0,timestamp)
            
        # Update data_arbitrage
        self.Update_Data_Arbitrage()
        
        
        return self.data_arbitrage

    
    def Check_Arbitrage_Hb_To_Bnc(self,i_hb,i_bnc,timestamp):
        # Check if Ask of Huobi with fees is lower than Bid of Binance
        if (self.Ask_hb[i_hb]*self.Ask_Fees) < (self.Bid_bnc[i_bnc]*self.Bid_Fees):
            # Check if we can sell the Huobi Volume on Binance
            if self.Ask_Volume_hb[i_hb] < self.Bid_Volume_bnc[i_bnc]:
                # Set Profits
                profits                                           = (self.Bid_bnc[i_bnc]*self.Bid_Fees) - (self.Ask_hb[i_hb]*self.Ask_Fees)
                
                if self.scenario==2 and  randint(0, 10) <= 2:
                    profits = -profits
                    
                # Push data into "data_arbitrage"
                self.data_arbitrage.loc[len(self.data_arbitrage)] = [timestamp,self.Ask_hb[i_hb],self.Bid_bnc[i_bnc],"Huobi",profits,self.Ask_Volume_hb[i_hb],profits*self.Ask_Volume_hb[i_hb]]
                
                # In we sell all the Huobi volume af first Ask we gonna check the next Huobi Ask 
                if (i_hb+1) < len(self.Ask_Volume_bnc):
                    # Update Volume
                    self.Bid_Volume_bnc[i_bnc]                    = self.Bid_Volume_bnc[i_bnc] - self.Ask_Volume_hb[i_hb]
                    # Recursive Call
                    self.Check_Arbitrage_Hb_To_Bnc(i_hb+1,i_bnc,timestamp)
            
            # if we cant sell all the volume, we gonna check the next bid of binance  
            elif (i_bnc+1) < len(self.Bid_Volume_hb):
                # Set Profits
                profits                                           = (self.Bid_bnc[i_bnc]*self.Bid_Fees) - (self.Ask_hb[i_hb]*self.Ask_Fees)
                if self.scenario==2 and randint(0, 10) <= 2:
                    profits = -profits
                # Push data into "data_arbitrage"
                self.data_arbitrage.loc[len(self.data_arbitrage)] = [timestamp,self.Ask_hb[i_hb],self.Bid_bnc[i_bnc],"Huobi",profits,self.Bid_Volume_bnc[i_bnc],profits*self.Bid_Volume_bnc[i_bnc]]
                # Update Volume
                self.Ask_Volume_hb[i_hb]                          = self.Ask_Volume_hb[i_hb] - self.Bid_Volume_bnc[i_bnc]
                # Recursive Call
                self.Check_Arbitrage_Hb_To_Bnc(i_hb,i_bnc+1,timestamp)
    
    
    def Check_Arbitrage_Bnc_To_Hb(self,i_hb,i_bnc,timestamp):
        if (self.Ask_bnc[i_bnc]*self.Ask_Fees) < (self.Bid_hb[i_hb]*self.Bid_Fees):
            if self.Ask_Volume_bnc[i_bnc] < self.Bid_Volume_hb[i_hb]:
                # Set Profits
                profits                                           = (self.Bid_hb[i_hb]*self.Bid_Fees) - (self.Ask_bnc[i_bnc]*self.Ask_Fees)
                if self.scenario==2 and randint(0, 10) <= 2:
                    profits = -profits
                # Push data into "data_arbitrage"
                self.data_arbitrage.loc[len(self.data_arbitrage)] = [timestamp,self.Ask_bnc[i_bnc],self.Bid_hb[i_hb],"Binance",profits,self.Ask_Volume_bnc[i_bnc],profits*self.Ask_Volume_bnc[i_bnc]]
                
                if (i_bnc+1) < len(self.Ask_Volume_bnc):
                    # Update Volume
                    self.Bid_Volume_hb[i_hb]                      = self.Bid_Volume_hb[i_hb] - self.Ask_Volume_bnc[i_bnc]
                    
                    # Recursive Call
                    self.Check_Arbitrage_Bnc_To_Hb(i_hb,i_bnc+1,timestamp)
                
            elif (i_hb+1) < len(self.Bid_Volume_hb):
                # Set Profits
                profits                                           = (self.Bid_hb[i_hb]*self.Bid_Fees) - (self.Ask_bnc[i_bnc]*self.Ask_Fees)
                if self.scenario==2 and randint(0, 10) <= 2:
                    profits = -profits
                ## Push data into "data_arbitrage"
                self.data_arbitrage.loc[len(self.data_arbitrage)] = [timestamp,self.Ask_bnc[i_bnc],self.Bid_hb[i_hb],"Binance",profits,self.Bid_Volume_hb[i_hb],profits*self.Bid_Volume_hb[i_hb]]
                # Update Volume
                self.Ask_Volume_bnc[i_bnc]                        = self.Ask_Volume_bnc[i_bnc] - self.Bid_Volume_hb[i_hb]
                # Recursive Call
                self.Check_Arbitrage_Bnc_To_Hb(i_hb+1,i_bnc,timestamp)

     
    def Update_Data_Arbitrage(self):
        # Sort Data_Arbitrage
        self.data_arbitrage.sort_values(by='timestamp',inplace=True)
        # Create new columns with Cumulative Profits
        self.data_arbitrage['Cumulative of Total Profits in USDT'] = self.data_arbitrage['Total Profits in USDT'].cumsum()
        
        self.data_arbitrage.reset_index(drop=True,inplace=True)
        
        # Add 'returns' columns to 'bar_hb'
        self.bar_hb['returns']                                     = (self.bar_hb['close'].shift(-1) - self.bar_hb['close'])/self.bar_hb['close']
        # Add 'returns' columns to 'bar_bnc'
        self.bar_bnc['returns']                                    = (self.bar_bnc['close'].shift(-1) - self.bar_bnc['close'])/self.bar_bnc['close']
        
        
    def Display_Profits_fees_limit(self):
        # Set Title
        title_profit_fees               = "Profit (in USDT) versus Fees (in %)"
        title_nb_trades_fees            = "Nb Arbitrages versus Fees (in %)"
        title_sharpe_ratio_versus_fees  = "Sharpe Ratio Vs Fees"
        title_sortino_ratio_versus_fees = "Sortino Ratio Vs Fees"
        
        # Set graph
        fig                       = make_subplots(rows           = 4,
                                                  cols           = 1,
                                                  subplot_titles = [title_profit_fees, title_nb_trades_fees, title_sharpe_ratio_versus_fees, title_sortino_ratio_versus_fees])


        # Add first graph
        profits = [self.epocs[i]['Cumulative of Total Profits in USDT'].iloc[-1] for i in range(0,len(self.epocs))]
        fig.add_trace(
                    go.Scatter(x          = self.epocs_fees,
                               y          = profits,
                               name       = 'Total profits Vs Fees'),
                               row        = 1,
                               col        = 1
                )
        
        nb_trades = [len(self.epocs[i]['Cumulative of Total Profits in USDT']) for i in range(0,len(self.epocs))]
        fig.add_trace(
                    go.Scatter(x          = self.epocs_fees,
                               y          = nb_trades,
                               name       = 'Nb Arbitrages Vs Fees'),
                               row        = 2,
                               col        = 1
                )
        
        fig.add_trace(
                    go.Scatter(x          = self.epocs_fees,
                               y          = self.epocs_sharpe_ratio,
                               name       = 'Sharpe Ratio Vs Fees'),
                               row        = 3,
                               col        = 1
                )
        
        fig.add_trace(
                    go.Scatter(x          = self.epocs_fees,
                               y          = self.epocs_sortino_ratio,
                               name       = 'Sortino Ratio Vs Fees'),
                               row        = 4,
                               col        = 1
                )

    


        # Update axis Name, Size, Color
        fig.update_yaxes(title_text='Total Profits (in USDT)', row=1, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_yaxes(title_text='Nb Arbitrages', row=2, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_yaxes(title_text='Sharpe Ratio', row=3, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_yaxes(title_text='Sortino Ratio', row=4, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        
        fig.update_xaxes(title_text='Fees (in %)', row=1, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_xaxes(title_text='Fees (in %)', row=2, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_xaxes(title_text='Fees (in %)', row=3, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_xaxes(title_text='Fees (in %)', row=4, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        
        # Update tiles Size, Color
        fig.update_annotations(font=dict(family="Helvetica", size=12, color='black'))
        
        # Change charts size
        fig.update_layout(
                            autosize=False,
                            width=1000,
                            height=1000,
                        )

        return fig
        
    def Display_Profits(self):
        
        # Set title
        title_fig_1               = "Cumulative profits (in USDT) - Fees = "     + str(self.Fees*100) + "%"
        title_fig_2               = "Profits distribution (in USDT) - Fees = "   + str(self.Fees*100) + "%"
        title_fig_3               = "Huobi return distribution (in USDT)"
        title_fig_4               = "Binance return distribution (in USDT)"
        
        
        # Set graph
        fig                       = make_subplots(rows           = 4,
                                                  cols           = 1,
                                                  subplot_titles = [title_fig_1,title_fig_2,title_fig_3,title_fig_4])
        # Add first graph
        fig.add_trace(
            go.Scatter(x          = self.data_arbitrage['timestamp'].values,
                       y          = self.data_arbitrage['Cumulative of Total Profits in USDT'].astype(float),
                       name       = 'Cumulative Profits'),
                       row        = 1,
                       col        = 1
        )
        # Add Second graph
        fig.add_trace(
            go.Histogram(x        = self.data_arbitrage['Total Profits in USDT'].astype(float),
                         name     = 'Profits Distribution',
                         autobinx = True),
                         row      = 2,
                         col      = 1
        )
        # Add third graph
        fig.add_trace(
            go.Histogram(x        = self.bar_hb['returns'],
                         name     = 'Huobi return Distribution',
                         autobinx = True),
                         row      = 3,
                         col      = 1
        )
        # Add fourth graph
        fig.add_trace(
            go.Histogram(x        = self.bar_bnc['returns'],
                         name     = 'Binance return Distribution',
                         autobinx = True),
                         row      = 4,
                         col      = 1
        )
        
        
        # Update axis Name, Size, Color
        fig.update_yaxes(title_text='Cumulative profits (in USDT)', row=1, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_yaxes(row=2, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_yaxes(row=3, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_yaxes(row=4, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        
        fig.update_xaxes(title_text='timestamp', row=1, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_xaxes(title_text='Profits distribution (in USDT)', row=2, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_xaxes(title_text='Huobi return distribution (in USDT)', row=3, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        fig.update_xaxes(title_text='Binance return distribution (in USDT)', row=4, col=1, title_font=dict(size=10, color='black'), tickfont=dict(size=8))
        
        # Update tiles Size, Color
        fig.update_annotations(font=dict(family="Helvetica", size=12, color='black'))
        
        # Change charts size
        fig.update_layout(
                            autosize=False,
                            width=1000,
                            height=1000,
                        )
        
        return fig
    
    def Export_Data(self, name):
        name = name + ".csv"
        # Export data to csv
        self.data_arbitrage.to_csv(name)
    
    def Check_Metrics(self):
        print("-- Huobi Metrics --",
              "\n\tHistorical Mean return : " + str(np.mean(self.bar_hb['returns'])),
              "\n\tHistorical Volatility : "  + str(np.std(self.bar_hb['returns'])),
              "\n-- Binance Metrics --",
              "\n\tHistorical Mean return : " + str(np.mean(self.bar_bnc['returns'])),
              "\n\tHistorical Volatility : "  + str(np.std(self.bar_bnc['returns'])))

In [13]:
def run_scenario_(orderbook_hb, orderbook_bnc, bar_hb,bar_bnc, scenario):
    # Init Backtesting Class
    bb = Backtesting(orderbook_hb,orderbook_bnc,bar_hb,bar_bnc,scenario)
    # Run BacktestFunction and display output DataFrame "data_arbitrage"
    display(bb.BacktestFunction())
    # Display graphs
    display(bb.Display_Profits())
    # Display(Metrics)
    bb.Check_Metrics()
    # Export Data Arbritage
    name = 'scenario_' + str(scenario)
    bb.Export_Data(name)
    
    # Run BacktestFunction to get the limit fees and their impacts on the performances
    bb.run_epocs_fees_limit()
    # Display graph from Backtest fees Function
    display(bb.Display_Profits_fees_limit())

In [15]:
if __name__ == '__main__':
    scenario = 2
    run_scenario_(orderbook_hb, orderbook_bnc, bar_hb,bar_bnc, scenario)

,timestamp,Ask,Bid,By on,Profits by Link,Volume,Total Profits in USDT,Cumulative of Total Profits in USDT
0,2021-08-20 00:00:01,27.0947,27.0950,Huobi,0.0003,2.990,0.000897,0.000897
1,2021-08-20 00:00:02,27.0938,27.0950,Huobi,0.0012,32.180,0.038616,0.039513
2,2021-08-20 00:00:02,27.0947,27.0950,Huobi,0.0003,2.990,0.000897,0.040410
3,2021-08-20 00:00:04,27.0924,27.0930,Huobi,-0.0006,1.079,-0.000647,0.039763
4,2021-08-20 00:00:08,27.0950,27.0956,Binance,0.0006,0.370,0.000222,0.039985
...,...,...,...,...,...,...,...,...
2278,2021-08-20 01:59:07,27.2328,27.2330,Huobi,0.0002,19.990,0.003998,82.571932
2279,2021-08-20 01:59:08,27.2328,27.2330,Huobi,-0.0002,19.990,-0.003998,82.567934
2280,2021-08-20 01:59:34,27.2461,27.2470,Huobi,0.0009,15.070,0.013563,82.581497
2281,2021-08-20 01:59:47,27.2546,27.2560,Huobi,-0.0014,0.062,-0.000087,82.581410


-- Huobi Metrics -- 
	Historical Mean return : 1.4643865347555675e-06 
	Historical Volatility : 0.00029795876058763895 
-- Binance Metrics -- 
	Historical Mean return : 1.1710827337546865e-06 
	Historical Volatility : 0.00027651141332626266
Fees :  0 - Profit : 90.69239043017531
Fees :  1e-05 - Profit : 79.54264508084664
Fees :  2e-05 - Profit : 50.37571672762786
Fees :  3.0000000000000004e-05 - Profit : 53.839678209048245
Fees :  4e-05 - Profit : 31.231293509251405
Fees :  5e-05 - Profit : 35.70198724004978
Fees :  6e-05 - Profit : 29.43996543235863
Fees :  7.000000000000001e-05 - Profit : 17.152296615044868
Fees :  8e-05 - Profit : 15.844525254240306
Fees :  9e-05 - Profit : 10.813852399708884
Fees :  0.0001 - Profit : 17.870298167323483
Fees :  0.00011 - Profit : 11.008068278937182
Fees :  0.00012 - Profit : 4.347720053935391
Fees :  0.00013000000000000002 - Profit : 11.766999486730928
Fees :  0.00014000000000000001 - Profit : 7.001423925826496
Fees :  0.00015000000000000001 - Profi